## Import required libraries and packages

In [1]:
import yaml
import os
import pathlib
from tqdm import tqdm

from prompt import Prompt
from utils.ChatGPT import ChatGPT

In [ ]:
CONFIG_FILE = "./config/config.yaml"
OUTPUT_PATH = "../benchmark_dir"
AGENT_NAME = "ChatGPT" # Available: ChatGPT, Llama_3_1, Gemma_2, Mistral
MODEL_NAME = "gpt-4o-mini" # Available: gpt-4o, gpt-4o-mini, llama-3.1-70b-versatile, gemma2-9b-it, mixtral-8x7b-32768

## Helper functions

In [ ]:
def set_agent_configuration(configuration_file_path, agent_name):
    with open(configuration_file_path, mode="r") as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]
    # GPT-4o-mini
    if agent_name == "ChatGPT":
        agent_config["ChatGPT_1"]["api_keys"] = [os.environ["OPENAI_API_KEY"]]
    else:
        agent_config["Llaama_3_1"]["api_keys"] = [os.environ["GROQ_API_KEY"]]
        agent_config["Gemma_2"]["api_keys"] = [os.environ["GROQ_API_KEY"]]
        agent_config["Mistral"]["api_keys"] = [os.environ["GROQ_API_KEY"]]
    return agent_config

In [4]:
def create_path_if_non_existing(path):
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def save_prompt(prompt, test_input_file_path, agent_config, few_shot_count):
    prompt_filename = test_input_file_path.split('\\')[-1].split('.')[0] + '_prompt.txt'
    agent_name = agent_config["ChatGPT_1"]["model_name"]
    create_path_if_non_existing(f"{OUTPUT_PATH}/{agent_name}/prompts/{few_shot_count}_shot")
    with open(f"{OUTPUT_PATH}/{agent_name}/prompts/{few_shot_count}_shot/{prompt_filename}", "w") as file:
        file.write(prompt)

In [6]:
def save_gpt_response(predicted_instructions, test_input_file_path, agent_config, few_shot_count):
    gpt_response_filename = test_input_file_path.split('\\')[-1].split('.')[0] + "_gpt_response.yaml"
    gpt_response = {"gpt_response": predicted_instructions}
    agent_name = agent_config["ChatGPT_1"]["model_name"]
    create_path_if_non_existing(f"{OUTPUT_PATH}/{agent_name}/gpt_responses/{few_shot_count}_shot")
    with open(f"{OUTPUT_PATH}/{agent_name}/gpt_responses/{few_shot_count}_shot/{gpt_response_filename}", "w") as file:
        yaml.dump(gpt_response, file)

## Setup and Initializations

In [ ]:
agent_config = set_agent_configuration(configuration_file_path=CONFIG_FILE, agent_name=AGENT_NAME)

## Example for Few-shot Learning

In [ ]:
prompt = Prompt(
    agent_configuration=agent_config, agent_name=AGENT_NAME, model_name="llama"
)

In [14]:
few_shot_examples = prompt.create_few_shot_examples()

In [15]:
actual_prompt = prompt.create_actual_prompt(10)

In [16]:
from pprint import pprint
pprint(actual_prompt)

{'content': '- Sort(source="Sheet1!A2:G19", key1="Sheet1!C1", order="asc", '
            'orientation="column")\n'
            '\n'
            'Here is the supplementary documentation you can reference:\n'
            "['Sort(source: str, key1: str, order: str=\\'asc\\', orientation: "
            "str=\\'column\\')\\nArgs explanation:\\nsource (string): The "
            'range to sort.\\nkey1 (string): The key to sort by.\\norder '
            "(string): The order to sort by. It can be \\'asc\\' or "
            "\\'desc\\'.\\norientation (string): The orientation to sort by. "
            "It can be \\'column\\' or \\'row\\'.\\n\\nUsage example:\\n# "
            'Example 1: Sort the range (A1:E6) in Sheet1 by the first column '
            '(i.e. A column) in ascending order.\\nSort("Sheet1!A2:E6", '
            '"Sheet1!A1", "asc", "column") # Exculde the first row (i.e. '
            "A1:E1) because it is the header.\\n']\n"
            'Here is the corresponding sheet state:\n'

In [ ]:
no_of_examples = 10

In [ ]:
few_shot_examples = []
for example_index in range(no_of_examples):
    # example_src_file_path = excel_file_paths[example_index]
    # example_input_file_path = input_file_paths[example_index]

    # path_generator = create_path_generator(
    #     file_path=f"{OUTPUT_PATH}/intermediate_responses/"
    # )
    # correct_response_file_paths = [str(path) for path in path_generator]

    # example_correct_file_path = correct_response_file_paths[example_index]

    # example_sheet_state = get_sheet_state(
    #     file_path=example_src_file_path, backend=xw_backend
    # )

    example = (
        "USER\n"
        "{input_example}\n"
        "Here is the supplementary documentation you can reference:\n"
        "{documentation_example}\n"
        "Here is the corresponding sheet state:\n"
        "{sheet_state_example}\n\n"
        "ASSISTANT\n"
        "{correct_example}\n"
    )

    example = example.format(
        input_example=get_input_functions(example_input_file_path),
        documentation_example=extract_docs_for_input_functions(
            input_file_path=example_input_file_path,
            agent_config=agent_config,
            api_doc=api_doc,
        ),
        correct_example=get_correct_summarization(
            correct_file_path=example_correct_file_path
        ),
        sheet_state_example=get_sheet_state(
            file_path=example_src_file_path, backend=xw_backend
        ),
    )
    few_shot_examples.append(example)

## Test set (except one-shot example)

In [ ]:
async def call_chat_gpt(prompt):
    try:
        chatbot = ChatGPT(agent_config["ChatGPT_1"], context=[], interaction_mode=True)
        response = await chatbot(prompt)
    except Exception as e:
        print(f"error occurs when parsing response: {e}")
    else:
        return response

In [ ]:
for test_index in tqdm(range(11, 21)):
    test_input_file_path = input_file_paths[test_index]
    test_src_file_path = excel_file_paths[test_index]

    # Create prompt
    prompt = (
        "SYSTEM\n"
        "Summarize the each sub-step of instructions into explanations in natural language. "
        "Be brief and do not provide verbose explanations."
        "Avoid redundant steps and provide minimal steps\n\n"
        "{few_shot_examples}\n"
        "USER\n"
        "{actual_input}\n"
        "Here is the supplementary documentation you can reference:\n"
        "{actual_documentation}\n"
        "Here is the corresponding sheet state:\n"
        "{actual_sheet_state}\n"
    )

    # Format the prompt
    prompt = prompt.format(
        few_shot_examples = "\n".join(few_shot_examples),
        actual_input=get_input_functions(test_input_file_path),
        actual_documentation=extract_docs_for_input_functions(
            input_file_path=test_input_file_path,
            agent_config=agent_config,
            api_doc=api_doc,
        ),
        actual_sheet_state=get_sheet_state(
            file_path=test_src_file_path, backend=xw_backend
        ),
    )

    save_prompt(prompt, test_input_file_path, agent_config, no_of_examples)

    # Get GPT response
    response = await call_chat_gpt(prompt)

    predicted_instructions = response.split("\n")
    predicted_instructions = [instruction[2:] for instruction in predicted_instructions]
    save_gpt_response(predicted_instructions, test_input_file_path, agent_config, no_of_examples)